In [1]:
# Import the necessary libraries
import warnings
warnings.filterwarnings("ignore")

# data transformation
import pandas as pd
import numpy as np
from functools import reduce

# plotting
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import seaborn as sns
from matplotlib import gridspec
from pylab import *
import matplotlib.patches as mpatches
from PIL import Image

# map plotting
#import geopandas as gpd
from shapely.geometry import Point
from mpl_toolkits.axes_grid1 import make_axes_locatable

# classification
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics as mt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

# statistics
import scipy.stats as stats

#for generating date_time 
import time
import datetime

In [2]:
# The data set contains a variety of data types.  It is best to specify them before importing.
# KEY: proper name
# VALUE: pandas dtype
datatypes = {
    "Borough": "category",
    "Map Atlas": "category",
    "Block": "category",
    "Lot": "category",
    "Address": "str",
    "Parcel Name": "str",
    "Agency": "category",
    "Current Uses": "str",
    "Total Area": "int",
    "Open Petroleum Spill": "category",
    "Govt Clean-Up Program": "category",
    "Structure Completed": "float",
    "Number Structures": "int",
    "Total Gross Area Structures": "int",
    "Ratio Building to Floor Area": "float",
    "Allowable Building to Floor Area": "float",
    "Land Use Category": "category",
    "Community Board": "category",
    "Census Tract": "category",
    "Census Block": "category",
    "School Dist": "category",
    "Council District": "category",
    "Postcode": "category",
    "Fire Comp": "category",
    "Health Area": "category",
    "Health Ctr": "category",
    "Police Prct": "category",
    "Major Use": "category",
    "Number of Easements": "int",
    "Commercial Floor Area": "int",
    "Residential Floor Area": "int",
    "Office Floor Area": "int",
    "Retail Floor Area": "int",
    "Garage Floor Area": "int",
    "Storage Floor Area": "int",
    "Factory Floor Area": "int",
    "Other Floor Area": "int",
    "Num Floors": "float",
    "Residential Units": "int",
    "Residential and Non-Residential Units": "int",
    "Lot Front": "float",
    "Lot Depth": "float",
    "Bldg Front": "float",
    "Bldg Depth": "float",
    "Proximity Code": "category",
    "Irr Lot Code": "category",
    "Lot Type Code": "category",
    "Bsmt Code": "category",
    "Assess Land": "float",
    "Exempt Land": "float",
    "Exempt Tot": "float",
    "Year Alter 1": "float",
    "Year Alter 2": "float",
    "His Dist": "category",
    "Landmark": "str",
    "Condominium Number": "category",
    "Coordinates": "str",
    "E-Designation Number": "category",
    "Industrial Business Zone": "category",
    "Zone Dist": "category",
    "Zone Dist 2": "category",
    "Overlay 1": "category",
    "Overlay 2": "category",
    "SP Dist 1": "category",
    "SP Dist 2": "category",
    "Potential Urban Ag": "category",
    "Contact": "str",
    "EDC % Occupied": "float",
    "Pluto Version": "category",
    "Latitude": "float",
    "Longitude": "float",
    "BIN": "str",
    "NTA": "category"
}

# Similarly, each column will have to have a different method of handling missing values.
# KEY: proper name
# VALUE: missing value type
nan = np.nan
nas = {
    "Borough": "",
    "Map Atlas": "",
    "Block": nan,
    "Lot": nan,
    "Address": "",
    "Parcel Name": "",
    "Agency": "",
    "Current Uses": "",
    "Total Area": nan,
    "Open Petroleum Spill": "",
    "Govt Clean-Up Program": "",
    "Structure Completed": nan,
    "Number Structures": nan,
    "Total Gross Area Structures": nan,
    "Ratio Building to Floor Area": nan,
    "Allowable Building to Floor Area": nan,
    "Land Use Category": "",
    "Community Board": "",
    "Census Tract": nan,
    "Census Block": "",
    "School Dist": "",
    "Council District": "",
    "Postcode": "",
    "Fire Comp": "",
    "Health Area": "",
    "Health Ctr": "",
    "Police Prct": "",
    "Major Use": "",
    "Number of Easements": nan,
    "Commercial Floor Area": nan,
    "Residential Floor Area": nan,
    "Office Floor Area": nan,
    "Retail Floor Area": nan,
    "Garage Floor Area": nan,
    "Storage Floor Area": nan,
    "Factory Floor Area": nan,
    "Other Floor Area": nan,
    "Num Floors": nan,
    "Residential Units": nan,
    "Residential and Non-Residential Units": nan,
    "Lot Front": nan,
    "Lot Depth": nan,
    "Bldg Front": nan,
    "Bldg Depth": nan,
    "Proximity Code": "",
    "Irr Lot Code": "",
    "Lot Type Code": "",
    "Bsmt Code": "",
    "Assess Land": nan,
    "Exempt Land": nan,
    "Exempt Tot": nan,
    "Year Alter 1": nan,
    "Year Alter 2": nan,
    "His Dist": "",
    "Landmark": "",
    "Condominium Number": nan,
    "Coordinates": "",
    "E-Designation Number": "",
    "Industrial Business Zone": "",
    "Zone Dist": "",
    "Zone Dist 2": "",
    "Overlay 1": "",
    "Overlay 2": "",
    "SP Dist 1": "",
    "SP Dist 2": "",
    "Potential Urban Ag": "",
    "Contact": "",
    "EDC % Occupied": nan,
    "Pluto Version": "",
    "Latitude": nan,
    "Longitude": nan,
    "BIN": "",
    "NTA": ""
}
# Read in the data.
nyc = pd.read_csv(
    "./Data/city-owned-and-leased-property-local-law-48-of-2011.csv",
    dtype=datatypes,
    na_values=nas)

# Change the type of Structure Complete, Year Alter 1 and Year Alter 2
nyc["Structure Completed"] = nyc["Structure Completed"].fillna(0)
nyc["Year Alter 1"] = nyc["Year Alter 1"].fillna(0)
nyc["Year Alter 2"] = nyc["Year Alter 2"].fillna(0)
nyc["Structure Completed"] = nyc["Structure Completed"].apply(pd.to_numeric, downcast='integer')
nyc["Year Alter 1"] = nyc["Year Alter 1"].apply(pd.to_numeric, downcast='integer')
nyc["Year Alter 2"] = nyc["Year Alter 2"].apply(pd.to_numeric, downcast='integer')

# Preview the data
#nyc.head(10)

In [3]:
WIDTH = 15
HEIGHT = 15

# create proper names to label graphs
proper_names = list(nyc)

#replace space with underscore
nyc.columns = nyc.columns.str.replace(' ', '_')

#make lowercase
nyc.columns = nyc.columns.str.lower()

#create proper name dictionary for plotting
proper_name_dict = dict(zip(list(nyc.columns), proper_names)) 

# Lab 2: Classification

_You are to build upon the predictive analysis (classification) that you already completed in the
previous mini-project, adding additional modeling from new classification algorithms as well as
more explanations that are inline with the CRISP-DM framework. You should use appropriate cross
validation for all of your analysis (explain your chosen method of performance validation in detail).
Try to use as much testing data as possible in a realistic manner (you should define what you think
is realistic and why)._

_This report is worth 20% of the final grade. Please upload a report (one per team) with all code
used, visualizations, and text in a single document. The format of the document can be PDF,
*.ipynb, or HTML. You can write the report in whatever format you like, but it is easiest to turn in the
rendered Jupyter notebook. The results should be reproducible using your report. Please carefully
describe every assumption and every step in your report._

**Dataset Selection**
_Select a dataset identically to the way you selected for the first project work week and mini-project.
You are not required to use the same dataset that you used in the past, but you are encouraged.
You must identify two tasks from the dataset to regress or classify. That is:_

* _two classification tasks OR_
* _two regression tasks OR_
* _one classification task and one regression task_

_For example, if your dataset was from the diabetes data you might try to predict two tasks: (1)
classifying if a patient will be readmitted within a 30 day period or not, and (2) regressing what the
total number of days a patient will spend in the hospital, given their history and specifics of the
encounter like tests administered and previous admittance._

## Data Preparation

* **[10 points]** _Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis._
* **[5 points]** _Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created)._

### Separating the Data by Date

In section 1.2.1.2 we restricted the data to only those entries more recent than January 1st, 2017.  This time we would like to analyze if there is a statistically significant difference between the Assessed Land Value and Zone District 1 from year to year.  If there is, we will need to proceed taking into consideration the serial correlation associated with time series data.  If not, we will examine the year which contains the fewest missing values within the data.

In [4]:
# Convert the "Date Created" column to datetime format
nyc.date_created = pd.to_datetime(nyc['date_created'])

# Subset the data by date
nyc_2011 = nyc.loc[nyc.date_created < pd.to_datetime("1/11/2012")].copy()
nyc_2013 = nyc.loc[(nyc.date_created >= pd.to_datetime("1/1/2013")) & 
                   (nyc.date_created < pd.to_datetime("1/11/2014"))].copy()
nyc_2015 = nyc.loc[(nyc.date_created >= pd.to_datetime("1/1/2015")) & 
                   (nyc.date_created < pd.to_datetime("1/11/2016"))].copy()
nyc_2017 = nyc.loc[nyc.date_created >= pd.to_datetime("1/1/2017")].copy()

### Assessed Land ANOVA

Now that we've broken the data into four distinct groups according to their Date Created feature, let's see if there is any significant difference between the Assessed Land Value over the four years of collected data.  We will conduct an ANOVA test with:

_$H_{0}$: There is no difference in the Assessed Land Values for the four years of collected data._<br>
_$H_{a}$: The Assessed Land Value is different for at least one year of collected data._

In [5]:
# Conduct an ANOVA on the Assessed Land Value dropping missing observations
f_value, p_value = stats.f_oneway(nyc_2011.assess_land.dropna(), 
                                  nyc_2013.assess_land.dropna(), 
                                  nyc_2015.assess_land.dropna(), 
                                  nyc_2017.assess_land.dropna())
print("F-value: ", round(f_value, 4))
print("p-value: ", round(p_value, 4))

F-value:  0.1085
p-value:  0.9552


It is clear from the small F-value (.1085) and high p-value (.9552) that we fail to reject _$H_{0}$_ and conclude that there is no difference in the Assessed Land Value from year to year.  Because of this fact, we will not consider time series data and instead use the year with the fewest missing values contained within its data.

In [6]:
print("Total number of missing values from 2011: ", sum(sum(nyc_2011.isna())))
print("Total number of missing values from 2013: ", sum(sum(nyc_2013.isna())))
print("Total number of missing values from 2015: ", sum(sum(nyc_2015.isna())))
print("Total number of missing values from 2017: ", sum(sum(nyc_2017.isna())))

Total number of missing values from 2011:  256176
Total number of missing values from 2013:  230934
Total number of missing values from 2015:  251047
Total number of missing values from 2017:  247675


It would appear that the data from 2013 contains 16,741 fewer missing values than its next closest competitor, 2017.  For the regression task, we will proceed with the 2013 data.

### Zone District 1 $\chi^{2}$ Test

While there is no difference in the Assessed Land Value from year to year, we should also test to see if this is the case with Zone District 1.  The best way to do this is via $\chi^{2}$ test.  The four year categories will represent the columns for the test, and the unique district zoning codes will constitute the rows.  In order to ensure an apples to apples comparison, we must make sure that each year of data has the same number of unique codes, and replace missing values with 0's since there would be none of that code if it were missing from the entire year.

In [7]:
# The original nyc data set contains all of the unique values for zoning district
# Using this fact we can make sure each year contains the same series index
zd_obs = pd.DataFrame(data = {"year_2011":nyc_2011.zone_dist_1.value_counts(), 
                              "year_2013":nyc_2013.zone_dist_1.value_counts(),
                              "year_2015":nyc_2015.zone_dist_1.value_counts(),
                              "year_2017":nyc_2017.zone_dist_1.value_counts()},
                      index = nyc.zone_dist_1.value_counts().index)

# Now make sure any missing values are replaced with 0's.
zd_obs = zd_obs.fillna(0)

# Determine the expected frequencies
zd_exp = pd.DataFrame(data = stats.contingency.expected_freq(zd_obs),
                      columns = ["year_2011", "year_2013", "year_2015", "year_2017"],
                      index = nyc.zone_dist_1.value_counts().index)

# Conduct the Chi-square test
chi_value, p_value = stats.chisquare(zd_obs, zd_exp, axis = 1)
zd_p = pd.DataFrame(data = p_value,
                   columns = ["p_value"],
                   index = nyc.zone_dist_1.value_counts().index)

# Output the index labels where we reject the null hypothesis
print(zd_p[zd_p.p_value <= .05].sort_values(by = "p_value"))

             p_value
PARKUS  6.662681e-27
R1-2    1.248756e-12
R5B     3.734167e-10
R6A     6.083213e-08
R3X     1.897966e-05
R6      2.709212e-05
R3A     4.982276e-05
M2-3    5.848091e-05
PARK    3.692810e-04
C6-7    6.658455e-04
R5      1.694733e-03
R3-1    2.544257e-03
C4-4L   4.384525e-03
R6B     6.769558e-03
R7-2    7.063688e-03
C2-8    7.871353e-03
BPC     1.309845e-02
R7A     1.545017e-02
C2-7    2.498833e-02
R5D     2.901622e-02
C4-5    3.231522e-02


The preceeding output represents the p-values for those zoning codes in which we reject $H_{0}$ at the $\alpha$ = 0.05 significance level.  Out of the 146 unique zoning codes, only the above 21 had statistically significantly deviation from their expected values; and we fail to reject the null hypothesis for the other 125 codes and conclude that they do not vary significantly from year to year.  What is interesting is the remarkably low p-value for the zoning code "PARKUS."  This should certainly be investigated further to gain some understanding of why this particular zoning code is so whimsical over the years.

With 85.62% of zoning codes failing to reject $H_{0}$, it is probably safe to proceed with analyzing just the 2013 data for Zone Dist 1 as well.  We just have to recognize that any classification model would have to properly weight other features in order to capture the seemingly dynamic nature of the above 21 zoning codes for year data other than 2013.

### Prepare Class Variables

Recognizing that 2013 had the fewest number of missing values, we will now take a deeper look at the number of missing values from individual features.

In [8]:
# Get the percentage of missing values from the 2013 data
na_percent = (nyc_2013.isnull().sum() / nyc_2013.shape[0] * 100)

# change to dataframe for easier processing
na_percent = pd.DataFrame({
    'Feature': na_percent.index,
    'Percent Null': na_percent.values
})

# replace with proper name
na_percent = na_percent.replace(proper_name_dict)

# select only vlaues that are greater than 0. this makes the visual useful
na_percent = na_percent[na_percent['Percent Null'] > 0 ].sort_values(by='Percent Null',ascending=True)
na_percent

,Feature,Percent Null
8,Current Uses,0.080239
60,Zone Dist 1,0.218834
47,Lot Type Code,0.284485
48,Bsmt Code,0.284485
46,Irr Lot Code,0.284485
45,Proximity Code,0.284485
28,Major Use,0.284485
5,Address,0.299074
2,Map Atlas,0.342840
17,Land Use Category,1.597491


The table above displays all features that have at least one missing value (Percent Null > 0).  Features with greater than 70% missing values are too scarce to provide any meaningful insight, unless these attributes can be safely interpolated.  Additinally, the features Parcel Name, Map Atlas, Address, Agency, Postcode, Coordinates, Condominium Number, E-Designation Number, Pluto Version, Contact, BIN, NTA, Longitude and Lattitude which mostly are string datatypes consist primarily of unuseful information can be simply removed.

The Census Tract and Census Block features divide the boroughs and Census Tracts, respectively, into smaller sub-divisions for the purpose of 10 year census data.  These sub divisions would add hundreds (possibly thousands) of additional columns if they were one-hot encoded and convolute the classification task.  They too will be dropped.

In [9]:
# Reducing dimensionality (parcel name, address, agency, contact, postcode, coordinates, latitude,
# longitude, nta, e-designation number and condominium number) and also variables with over 80%
# missing values.
for col in ['parcel_name',
            'map_atlas',
            'address',
            'agency',
            'census_tract',
            'census_block',
            'sp_dist_1',
            'sp_dist_2',
            'postcode',
            'overlay_1',
            'overlay_2',
            'his_dist',
            'landmark',
            'health_area',
            'health_ctr',
            'zone_dist_2',
            'potential_urban_ag',
            'industrial_business_zone',
            'edc_%_occupied',
            'govt_clean-up_program',
            'open_petroleum_spill',
            'latitude',
            'longitude',
            'condominium_number',
            'coordinates',
            'e-designation_number',
            'pluto_version',
            'contact',
            'bin',
            'nta']:
    if col in nyc_2013: del nyc_2013[col]

In [10]:
# Drop observations with null values from the remaining features < 70% null
nyc_2013.dropna(subset = ['current_uses',
                          'zone_dist_1',
                          'irr_lot_code',
                          'bsmt_code', 
                          'lot_type_code',
                          'proximity_code',
                          'major_use',
                          'land_use_category',
                          'school_dist',
                          'fire_comp',],
                inplace=True)

The 2013 data has thus been reduced from 13,709 to 11,496 observations and 74 features down to 45 and contains no missing values.  Of the remaining features, the Structure Completed attribute identifies the year that the structure was completed on the lot.  We now engineer a new feature that provides a simple "Yes" if the structure was completed, or "No" if the year is 0.

In [11]:
# Create the new column
nyc_2013["structure_completed_2"] = nyc_2013.structure_completed

# Convert the type to string
nyc_2013.structure_completed_2 = nyc_2013.structure_completed_2.astype(str)

# Assign the data
nyc_2013.loc[nyc_2013.structure_completed == 0, "structure_completed_2"] = "No"
nyc_2013.loc[nyc_2013.structure_completed != 0, "structure_completed_2"] = "Yes"

# Convert back to categorical
nyc_2013.structure_completed_2 = nyc_2013.structure_completed_2.astype("category")

# Output the results
print(nyc_2013.structure_completed_2.value_counts())

No     7665
Yes    3831
Name: structure_completed_2, dtype: int64


Adding this feature provides an "at a glance" view if the lot has a completed structure on it.  Additionally, data older than 1985 rapidly becomes less reliable when determining the year completed, so this new feature will not concern itself with inaccuracies in the actual year of completion.

There are 137 categories for zone dist 1.  Let's create a new feature that reduces the number of categories to something easier.  The NYC Zoning Map is split into three main zoning districts: Commercial (C), Residential (R), and Manufacturing (M).  Within each of these districts, low, medium, and high-density districts are mapped out. (https://www.reonomy.com/blog/post/guide-to-nyc-zoning).  There is also another category so called PARK.  Hence, we divide our zoning districs into 4 categories.

In [12]:
# Just like as above...
nyc_2013["new_zone_dist"] = nyc_2013.zone_dist_1
nyc_2013.new_zone_dist = nyc_2013.new_zone_dist.astype(str)
# Only FANCIER!
for first_letter, zone in zip(["C", "R", "M", "P"], ["Commercial", "Residential", "Manufacturing", "Park"]):
    nyc_2013.loc[nyc_2013.zone_dist_1.astype(str).apply(lambda x: x[0]) == first_letter, "new_zone_dist"] = zone
nyc_2013.new_zone_dist = nyc_2013.new_zone_dist.astype("category")

#Output the results
print(nyc_2013.new_zone_dist.value_counts())

Residential      6858
Park             2551
Manufacturing    1281
Commercial        806
Name: new_zone_dist, dtype: int64


The New Zone Dist feature substantially reduces the number of categories in Zone District 1 from 137 down to 4.  Although we may potentially lose some useful information by omitting everything but the first letter of the zoning code, the intricacies within those extra characters would likely convolute any classification algorithm we attempt.

The Major Use feature contains a wealth of information, but has too many levels within it to be useful.  Appendix C of https://www1.nyc.gov/assets/planning/download/pdf/data-maps/open-data/pluto_datadictionary.pdf (pg. 49) outlines each individual code under the category it falls into.  Instead of using the entire code, we'll use just the first letter in the alphanumeric code.  While some information will be lost in this process, the first character is where the bulk of the information about this code lies.

In [13]:
# Just like as above...
nyc_2013["bldg_class"] = nyc_2013.major_use
nyc_2013.bldg_class = nyc_2013.bldg_class.astype(str)
# Now iterate through every letter in the alphabet, replacing the code with just the letter
for first_letter in [chr(i) for i in range(ord('A'),ord('Z')+1)]:
    nyc_2013.loc[nyc_2013.major_use.astype(str).apply(lambda x: x[0]) == first_letter, "bldg_class"] = first_letter
nyc_2013.bldg_class = nyc_2013.bldg_class.astype("category")

# We can safely drop the Major Use category now.

#Output the results
print(nyc_2013.bldg_class.value_counts().sort_index())

A      34
B      19
C     255
D      39
E      63
F      41
G     521
H       5
I      98
J       9
K      78
L       3
M      31
N      52
O     238
P     287
Q    2803
R       2
S      33
T     146
U     448
V    4569
W    1210
Y     436
Z      76
Name: bldg_class, dtype: int64


There is no category for "X" in the PLUTO Data Dictionary which is why we don't see any counts for it.  This step substantially reduces the number of columns that will be generated when we create dummy variables for each level in this categorical variable.

The last feature would like to revisit is the Assessed Land Value. We have already identified this feature as our target for regression, but by creating different bins constructed from the summary statistic quartiles, we can also look at this from a classification perspective. We create a new feature to do jus that.

In [14]:
# Create a new column that bins assess_land into 3 categories
nyc_2013["assess_range"] = pd.qcut(nyc.assess_land, 3, labels=['low', 'medium', 'high'])

# Display the results by new_zone_dist
nyc_grouped = nyc_2013.groupby(['new_zone_dist', 'assess_range'])
print(nyc_grouped.assess_range.count())

new_zone_dist  assess_range
Commercial     low               73
               medium           200
               high             533
Manufacturing  low              235
               medium           521
               high             525
Park           low              497
               medium           976
               high            1078
Residential    low             2757
               medium          2211
               high            1890
Name: assess_range, dtype: int64


The above output tells us that the majority of properties belong the the residential zoning code, and that every zoning code contains more "High" assessed land values than any other category within their group.  The last step in the Data Preparation phase is to one-hot encode our categorical variables.  Upon closer inspection, there are a few more columns that may be excluded from the final data set.  They are:

* Date Created
* Block
* Lot
* Current Uses
* Major Use (before the dimensionality reduction)
* Zone District 1 (before the dimensionality reduction)

In [15]:
# Identify categorical variables.  
# We leave out Land Use Category and Structure Completed 2 because we'll use it in a visual later
categorical_fields = list(nyc_2013.drop(
    columns = ["block", 
               "lot", 
               "major_use",
               "new_zone_dist",
               "zone_dist_1", 
               "assess_range", 
               "land_use_category",
               "structure_completed_2"]).select_dtypes(
    include=['category']).columns)

# Creating dummy variables for their levels
nyc_2013 = pd.get_dummies(nyc_2013, columns = categorical_fields).drop(
    columns = ["date_created", 
               "block", "lot", 
               "current_uses", 
               "major_use", 
               "zone_dist_1"])
    
print("Shape after one-hot encoding categorical variables: ", nyc_2013.shape)

Shape after one-hot encoding categorical variables:  (11496, 663)


In [16]:
# Save the data for ease of loading.
#nyc_2013.to_csv("./Data/nyc_2013.csv", index = False)

### Describe the Data

After the addition of the Assessed Land Value Range feature, we have a complete picture of our data set.  Summary statistics for all features are as follows:

In [17]:
# Describe the data
nyc_2013.describe(include = "all").transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
total_area,11496,NaN,NaN,NaN,154970,2.40516e+06,0,2008.75,6000,35841.5,2.14756e+08
structure_completed,11496,NaN,NaN,NaN,648.238,917.127,0,0,0,1926,2012
number_structures,11496,NaN,NaN,NaN,0.60047,2.52583,0,0,0,1,121
total_gross_area_structures,11496,NaN,NaN,NaN,34745.2,341706,0,0,0,6063.75,2.76e+07
ratio_building_to_floor_area,11496,NaN,NaN,NaN,0.634766,2.00386,0,0,0,0.41,59.89
allowable_building_to_floor_area,11496,NaN,NaN,NaN,1.55182,1.98763,0,0,0.6,2.43,12
land_use_category,11496,11,11,4569,NaN,NaN,NaN,NaN,NaN,NaN,NaN
number_of_easements,11496,NaN,NaN,NaN,0.0473208,0.295869,0,0,0,0,7
commercial_floor_area,11496,NaN,NaN,NaN,33039.6,337206,0,0,0,3200,2.76e+07
residential_floor_area,11496,NaN,NaN,NaN,1704.76,47140.6,0,0,0,0,3.97974e+06


### EDA on Reduced Data Set

Our data has taken on a completely new shape and contains 3 new features while eliminating 27 less than useful feature.  We now explore our new data to hopefully gain some deeper understanding before delving into classification and/or regression tasks.

In [18]:
# Define the figure and land use category labels
fig, ax = plt.subplots(2,2,figsize=(WIDTH, HEIGHT))
labels = ['One & Two Family Buildings', 
          'Parking Facilities',
          'Vacant Land',
          'Multi-Family Walk-Up Buildings',
          'Multi-Family Elevator Buildings',
          'Mixed Residential & Commercial Buildings',
          'Commercial & Office Buildings',
          'Industrial & Manufacturing',
          'Transportation & Utility',
          'Public Facilities & Institutions',
          'Open Space & Outdoor Recreation']

# Top plots
plt.suptitle('Zoning Distict & Land Use Category Breakdown')
ch1 = nyc_grouped['assess_range'].count().unstack('assess_range')
ch1[['low', 'medium', 'high']].plot(kind ='bar', stacked = True , ax = ax[0][0])
ax[0][0].set_title("Zone District")

nyc1_grouped2 = nyc_2013.groupby(["land_use_category", "assess_range"])
ch2 = nyc1_grouped2['assess_range'].count().unstack('assess_range')
ch2[['low', 'medium', 'high']].plot(kind ='bar', stacked=True , ax = ax[0][1])
ax[0][1].set_xticklabels((labels))
ax[0][1].set_title("Land Use Category")

# Bottom plots
fig.subplots_adjust(hspace = .6)
ch1_grouped = nyc_2013.groupby(['new_zone_dist', 'structure_completed_2'])
ch1 = ch1_grouped['structure_completed_2'].count().unstack('structure_completed_2')
ch1[['Yes', 'No']].plot(kind = 'bar', stacked=True, ax = ax[1][0])

ch2_grouped = nyc_2013.groupby(['land_use_category', 'structure_completed_2'])
ch2 = ch2_grouped['structure_completed_2'].count().unstack('structure_completed_2')
ch2[['Yes', 'No']].plot(kind = 'bar', stacked=True, ax = ax[1][1])
ax[1][1].set_xticklabels((labels))

plt.show()

The first thing that can be seen from the above plots are that the zone district plots (left plots) have nearly a uniform ratio between the three different assess value ranges and whether the structure is complete or not.  The Commercial zone appears to be the only code with any noticable deviation.  It kind of makes sense in terms of real estate that commerical property would be more heavily skewed to the expensive and that there is in fact a building front to do business!

The plots on the right are a wealth of knowledge.  Looking at the lengths of the bars and the distribution of colors, land use category looks like it would be a prime candidate for a decision tree classifier.  To look more in detail, let's look at some box plots.

In [19]:
# Define the figure and labels
fig, ax = plt.subplots(3,2, figsize=(WIDTH, HEIGHT))
plt.suptitle('Boxplot Analysis')
land_labels = ['One & Two Family Buildings', 
          'Multi-Family Walk-Up Buildings'
          ,'Multi-Family Elevator Buildings',
          'Mixed Residential & Commercial Buildings',
          'Commercial & Office Buildings',
          'Industrial & Manufacturing',
          'Transportation & Utility',
          'Public Facilities & Institutions',
          'Open Space & Outdoor Recreation',
          'Parking Facilities',
          'Vacant Land']
zone_labels = ['Commercial',
               'Manufacturing',
               'Park',
               'Residential']

nyc_2013.loc[nyc_2013.assess_land < 300000].boxplot(
    column=['assess_land'],
    by='land_use_category',
    ax=ax[0][0],
    vert=False
)
ax[0][0].set_yticklabels(land_labels)

nyc_2013.loc[nyc_2013.assess_land < 300000].boxplot(
    column=['assess_land'],
    by='new_zone_dist',
    ax=ax[0][1],
    vert=False
)
ax[0][1].set_yticklabels(zone_labels)

nyc_2013.loc[nyc_2013.assess_land < 600000].boxplot(
    column=['assess_land'],
    by='land_use_category',
    ax=ax[1][0],
    vert=False
)
ax[1][0].set_yticklabels(land_labels)

nyc_2013.loc[nyc_2013.assess_land < 600000].boxplot(
    column=['assess_land'],
    by='new_zone_dist',
    ax=ax[1][1],
    vert=False
)
ax[1][1].set_yticklabels(zone_labels)

nyc_2013.loc[nyc_2013.assess_land < 1000000].boxplot(
    column=['assess_land'],
    by='land_use_category',
    ax=ax[2][0],
    vert=False
)
ax[2][0].set_yticklabels(land_labels)

nyc_2013.loc[nyc_2013.assess_land < 1000000].boxplot(
    column=['assess_land'],
    by='new_zone_dist',
    ax=ax[2][1],
    vert=False
)
ax[2][1].set_yticklabels(zone_labels)

plt.show()

From the above left plot, we can see the zone districts and assessed land value ranges in more detail.  It can be seen that the assess land value of residential and manufacturing districts have a lower assessed value range when compared to parks and commercial zones.  The commerical district has the largest range of assess land values with no outliers outside of the Q3 + 1.5(IQR) range.  We also see that is has a much bigger median compared to the other districts which hold several data points in the upper outlier region (especially Residential zones).  As a result, we can conclude that there is an important relationship between assessed land value range and zone districts.  The Land Use Category plots on the left are a bit harder to interpret since many classes have numerous outliers and some of the data appears to have a median of 0 (Open Space & Outdoor Recreation < $300,000).

We now one-hot encode Land Use Category and Structure Completed 2 for use in our classification models.

In [20]:
# One-hot encode "Land Use Category" and "Structure Completed 2"
nyc_2013 = pd.get_dummies(nyc_2013, columns = ["land_use_category", "structure_completed_2"])
    
print("Final shape of data: ", nyc_2013.shape)

Final shape of data:  (11496, 674)


## Modeling and Evaluation

* **[10 points]** _Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions._
* **[10 points]** _Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time._
* **[20 points]** _Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!_
* **[10 points]** _Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model._
* **[10 points]** _Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course._
* **[10 points]** _Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task._

### Classification Task for New Zone Dist

This section will be dedicated to creating, training, and adjusting models designed to classify the New Zone District zoning code. We will models for Logistic Regression, K-Nearest Neighbors (KNN), and XGBoost for classifying this feature.

Let's start by examine the distribution of classes and then decide the best split.

In [ ]:
# Output the distribution of New Zone Dist labels
for i in list(range(0,4)):
    print(nyc_2013.new_zone_dist.value_counts().index[i] + ": ",
          nyc_2013.new_zone_dist.value_counts()[3-i], 
          " or",
          round(nyc_2013.new_zone_dist.value_counts()[3-i]
                /nyc_2013.new_zone_dist.value_counts().sum()
                *100, 2),
          "%")

As before, we see a similar situation where the bulk of the data is labeled in just one category ("Residential").  To start the classification task, we need to get our training data in the right format.  That is, the training data must exclude our target variable, New Zone District.

In [21]:
# Identify categorical variables
categorical_fields = list(nyc_2013.drop(
    columns = ["new_zone_dist"]).select_dtypes(
    include=['category']).columns)

# Creating dummy variables for their levels
X = pd.get_dummies(nyc_2013, columns = categorical_fields).drop(
    columns = ["new_zone_dist"])
    
print("Training data shape after one-hot encoding categorical variables: ", X.shape)

Training data shape after one-hot encoding categorical variables:  (11496, 675)


#### Logistic Regression Model

For the Logistic Regression (LR) model, we will adapt some of the findings from the Mini Lab (section 2.1.3) to make this one even better. Accuracy is still a good evaluation metric, but one of the main concerns last time was that the majority of the data fell into 2 of the 11 categories. To prevent the classification model from being too heavily biased in the direction of majority data, we will not employ the 80/20 train/test split that we did in 2.1.3. Instead, we'll try a Statified Split which will maintain the same proportion each label in the testing and training data.

In [22]:
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.5, train_size=0.5)

le = preprocessing.LabelEncoder()
nzd = le.fit_transform(nyc_2013.new_zone_dist)

In [23]:
#X = nyc_2013.drop(columns = ["new_zone_dist"])
for trainidx, testidx in cv.split(X, nzd):
    # note that these are sparse matrices
    X_train = X.iloc[trainidx] 
    X_test = X.iloc[testidx] 
    y_train = nzd[trainidx]
    y_test = nzd[testidx]

In [ ]:
logit_model = LogisticRegression(solver = 'sag', max_iter = 1000, multi_class= "multinomial")

In [ ]:
%%time
logit_model_result = logit_model.fit(X_train, y_train)
test_predictions = logit_model_result.predict(X_test)
train_predictions = logit_model_result.predict(X_train)
logit_accuracy = accuracy_score(y_test, test_predictions).round(2) * 100
print("LR Accuracy: ", logit_accuracy)

In [ ]:
# Define the figure
fig, ax = plt.subplots(1, 2, figsize=(WIDTH, HEIGHT))
cbar_ax = fig.add_axes([.91, .3, .03, .4])
ax[0].set_title('Training Data Results', fontdict={'size': 20})
ax[1].set_title('Testing Data Results', fontdict={'size': 20})
fig.text(.47, .25, 'New Zone District Predicted label', ha='center', va='center', fontdict={'size': 15})
fig.text(-.03, 0.5, 'New Zone District True Label', ha='center', va='center', rotation='vertical', fontdict={'size': 15})
# Labels for "Y" and "N" don't seem to load either.  Tried many different ways.
labels = le.classes_
print(labels)
mat1 = confusion_matrix(y_train, train_predictions)/len(y_train)*100
mat2 = confusion_matrix(y_test, test_predictions)/len(y_test)*100
ax[0] = sns.heatmap(mat1, 
            square=True, 
            annot=True,
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[0])
ax[0].set_xticklabels((labels))
ax[0].set_yticklabels((labels))
ax[1] = sns.heatmap(mat2, 
            square=True, 
            annot=True, 
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[1])
ax[1].set_xticklabels((labels))
ax[1].set_yticklabels((labels))
fig.tight_layout(rect=[0, 0, .9, 1])

Each cell in the above confusion matrix represents the percentage of data under the given classification.  We see that the LR model is not very accurate, incorrectly predicting Parks 45% of the time when it should really be Residential.  It seemed to perform best when classifying Residential, with a precision of .7489.

Since accuracy is our evaluation metric however, let's see if we can adjust some parameters of the model and obtain a better fit.  The first thing we can try is to increase the maximum number of iterations.  We'll increase it by a factor of 10 and see if this helps at all.

In [ ]:
%%time

# Increase max_iter to 10,000
logit_model = LogisticRegression(solver = 'sag', max_iter = 10000, multi_class= "multinomial")

# Train the model
logit_model_result = logit_model.fit(X_train, y_train)
test_predictions = logit_model_result.predict(X_test)
train_predictions = logit_model_result.predict(X_train)
logit_accuracy = accuracy_score(y_test, test_predictions).round(2) * 100
print("LR Accuracy: ", logit_accuracy)

After over 20 minutes of processing, we see that increasing the maximum number of iterations did not do anything to improve the accuracy of the LR model.  Instead, let's try using a different value for C with the original number of iterations (1000).

In [ ]:
%%time

# Use a low value for C
logit_model = LogisticRegression(solver = 'sag', C = .01, max_iter = 1000, multi_class= "multinomial")

# Train the model
logit_model_result = logit_model.fit(X_train, y_train)
test_predictions = logit_model_result.predict(X_test)
train_predictions = logit_model_result.predict(X_train)
logit_accuracy = accuracy_score(y_test, test_predictions).round(2) * 100
print("LR Accuracy: ", logit_accuracy)

Intuitively, a low C value should do a better job fitting the training data.  But when put up against the test data, we see this too had no effect on the overall accuracy.  Next we'll try a high value for C to see if a more "generalized" approach is best.

In [ ]:
%%time

# Use a high value for C
logit_model = LogisticRegression(solver = 'sag', C = 100, max_iter = 1000, multi_class= "multinomial")

# Train the model
logit_model_result = logit_model.fit(X_train, y_train)
test_predictions = logit_model_result.predict(X_test)
train_predictions = logit_model_result.predict(X_train)
logit_accuracy = accuracy_score(y_test, test_predictions).round(2) * 100
print("LR Accuracy: ", logit_accuracy)

It appears the value of C does not affect this model.  The last thing to try is a different solver.

In [ ]:
%%time

# Use the newton-cg model
logit_model = LogisticRegression(solver = 'saga', max_iter = 1000, multi_class= "multinomial")

# Train the model
logit_model_result = logit_model.fit(X_train, y_train)
test_predictions = logit_model_result.predict(X_test)
train_predictions = logit_model_result.predict(X_train)
logit_accuracy = accuracy_score(y_test, test_predictions).round(2) * 100
print("LR Accuracy: ", logit_accuracy)

The saga solver fails to deliver any improvement to test data accuracy.  We'll try Newton's Method as a last resort.

In [24]:
%%time

# Use the newton-cg model
logit_model = LogisticRegression(solver = 'newton-cg', max_iter = 1000, multi_class= "multinomial")

# Train the model
logit_model_result = logit_model.fit(X_train, y_train)
test_predictions = logit_model_result.predict(X_test)
train_predictions = logit_model_result.predict(X_train)
logit_accuracy = accuracy_score(y_test, test_predictions).round(2) * 100
print("LR Accuracy: ", logit_accuracy)

LR Accuracy:  69.0
CPU times: user 34min 34s, sys: 17 s, total: 34min 51s
Wall time: 5min 49s


/Users/austinsimeone/anaconda3/envs/env_7331/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


It seems the LR model does not discriminate against the maximum number of iterations or the value of C when determining accuracy.  Using the Newton-cg solver, however, seems to have nearly doubled the accuracy.  We'll examine the confusion matrix below to gain some insight on the classification process.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(WIDTH, HEIGHT))
cbar_ax = fig.add_axes([.91, .3, .03, .4])
ax[0].set_title('Training Data Results', fontdict={'size': 20})
ax[1].set_title('Testing Data Results', fontdict={'size': 20})
fig.text(.47, .25, 'New Zone District Predicted label', ha='center', va='center', fontdict={'size': 15})
fig.text(-.03, 0.5, 'New Zone District True Label', ha='center', va='center', rotation='vertical', fontdict={'size': 15})
# Labels for "Y" and "N" don't seem to load either.  Tried many different ways.
labels = le.classes_
print(labels)
mat1 = confusion_matrix(y_train, train_predictions)/len(y_train)*100
mat2 = confusion_matrix(y_test, test_predictions)/len(y_test)*100
ax[0] = sns.heatmap(mat1, 
            square=True, 
            annot=True,
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[0])
ax[0].set_xticklabels((labels))
ax[0].set_yticklabels((labels))
ax[1] = sns.heatmap(mat2, 
            square=True, 
            annot=True, 
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[1])
ax[1].set_xticklabels((labels))
ax[1].set_yticklabels((labels))
fig.tight_layout(rect=[0, 0, .9, 1])
plt.show()

Here we can see that the model now correctly classifies the majority of the data as Residential.  The precision for the Residential classification is only .6818, which is actually a step down from the first model we ran (.7489). Depending on the application of classifying, the original model might be better. But seeing as how accuracy is our metric for evaluating the model, we conclude that the LR model performs best with the Newton-cg solver.

The main advantage to the LR model is that there are several parameters to experiment with to find the best model.  Unfortunately for our data, many of those did not seem to have a large impact on the accuracy of the model.  Newton's Method performed significantly better than Stochasitc Average Gradient (SAG) Descent and its variant, SAGA, but the long processing time makes it cumbersome to vary additionaly parameters to truly achieve the best model possible.  There are no doubt other LR models that will produce a higher accuracy if you're willing to wait approximately 10 minutes per trial.  Instead, we will explore other classification models.  But before we move on, let's see if we can understand which features carried the most weight in final LR model we produced.

In [ ]:
# Get the weights for all features
impt_features = logit_model_result.coef_

# Create a placeholder for just the top 10 most important
top_impt = np.zeros( (impt_features.shape[0], 10), dtype = 'float')
indices = np.zeros( (impt_features.shape[0], 10), dtype = 'int')

# Define the figure
fig, ax = plt.subplots(2, 2, figsize=(WIDTH, HEIGHT))

# Extract the relevant data
for i in range(0, impt_features.shape[0]):
    indices[i,:] = impt_features[i, :].argsort()[:10]
    top_impt[i, :] = impt_features[i, indices[i, :]]
    #plt.xticks(range(0,10), list( X_train.columns[i] for i in impt_features[i, :].argsort()[:10]), rotation = 90)

# Plot it on a subplot
ax[0, 0].set_title('Commercial Features', fontdict={'size': 20})
ax[0, 0].bar(range(0, 10), top_impt[0, :])
ax[0, 0].xaxis.set_ticks(np.arange(0, 10))
ax[0, 0].set_xticklabels( list( X_train.columns[i] for i in indices[0, :]), rotation = 90)
ax[0, 1].set_title('Manufacturing Features', fontdict={'size': 20})
ax[0, 1].bar(range(len(top_impt[1, :])), top_impt[1, :])
ax[0, 1].xaxis.set_ticks(np.arange(0, 10))
ax[0, 1].set_xticklabels( list( X_train.columns[i] for i in indices[1, :]), rotation = 90)
fig.subplots_adjust(hspace = .6)
ax[1, 0].set_title('Park Features', fontdict={'size': 20})
ax[1, 0].bar(range(len(top_impt[2, :])), top_impt[2, :])
ax[1, 0].xaxis.set_ticks(np.arange(0, 10))
ax[1, 0].set_xticklabels( list( X_train.columns[i] for i in indices[2, :]), rotation = 90)
ax[1, 1].set_title('Residential Features', fontdict={'size': 20})
ax[1, 1].bar(range(len(top_impt[3, :])), top_impt[3, :])
ax[1, 1].xaxis.set_ticks(np.arange(0, 10))
ax[1, 1].set_xticklabels( list( X_train.columns[i] for i in indices[3, :]), rotation = 90)


plt.show()

Above we can see the weights of the top 10 most influential features in determining the given classification.  Exempt Land, Total Area, Commercial Floor Area, Residential Floor Area, and Total Gross Area Structures make appearances in 3 of the 4 classifications.  The actual values of the weights are output below.

In [ ]:
lst = (list( X_train.columns[i] for i in indices[0, :]),
list( X_train.columns[i] for i in indices[1, :]),
list( X_train.columns[i] for i in indices[2, :]),
list( X_train.columns[i] for i in indices[3, :]))


In [ ]:
flat_list = [item for sublist in lst for item in sublist]

In [ ]:
pd.Series(flat_list).value_counts()

In [ ]:
LR_weights = pd.DataFrame(top_impt, 
                          index = ["Commercial", "Manufacturing", "Park", "Residential"],
                          columns = ["Rank 1", "Rank 2", "Rank 3", "Rank 4", "Rank 5", 
                                     "Rank 6", "Rank 7", "Rank 8", "Rank 9", "Rank 10"])
LR_weights

For Residential, it is clear to see why Assessed Land Value carries so much weight.  Housing in NYC is unaffordable to most for a reason after all!  Likewise, it makes sense that when classifying a zone as a Park, you would want to look at the Exempt Total since Parks are unlikely to pay hefty taxes.  Similarly, if a lot has a small Total Area, I would lean to believe it to my a Commercial store front.  What is curious to find is the Bldg Class Q that occupies the rank 8 spot in Residential.  This is odd becuase Bldg Class code Q is for Outdoor Recreation Facilities.  Perhaps the model is trying to rule out Park by checking for this Bulding Class code.

#### KNN Model

For the KNN model to predict New Zone Distist, we will use accuracy and precision as our evaluation metric since we have four different classes and we would like to know the accuracy per class.  We will use the Statified 50/50 Split which will maintain the same proportion of each label in the testing and training data.  This will ensure that there isn't a biased distribution of labels from one classification.

In [25]:
# Define the split
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.5, train_size=0.5)

# Encode the labels
le = preprocessing.LabelEncoder()
nzd = le.fit_transform(nyc_2013.new_zone_dist)

# Split the data
for trainidx, testidx in cv.split(X, nzd):
    # note that these are sparse matrices
    X_train = X.iloc[trainidx] 
    X_test = X.iloc[testidx] 
    y_train = nzd[trainidx]
    y_test = nzd[testidx]

A common practice is to set K equal to the square root of the number of training samples for best results.  We'll test the KNN model for every value of K from 1 to $\sqrt n$ + 3 to find the K which gives us the best accuracy, where _n_ is the total number of training samples, 5,748.  The additional 3 beyond $\sqrt n$ is just to see if further values yield a better accuracy.

In [ ]:
%%time
KNNdf = pd.DataFrame(index = list(range(1, int(sqrt(X_train.shape[0]) + 3))),
                     columns = ["Accuracy"],
                     dtype = "float")
for i in range(1, len(KNNdf) + 1 ):
    clf = KNeighborsClassifier(n_neighbors = i)
    clf.fit(X_train, y_train)
    yhat= clf.predict(X_test)
    total_accuracy = mt.accuracy_score(y_test, yhat) * 100
    KNNdf.loc[i, "Accuracy"] = total_accuracy

print("Max accuracy is", 
      round(KNNdf.loc[KNNdf.Accuracy.idxmax(), "Accuracy"], 2), 
      "with", KNNdf.Accuracy.idxmax(), "neighbors.")

It turns out that the optimal number of neighbors is 48.  Had we used the $\sqrt n$ approach, we would have seen an accuracy of 66.13%, or a difference of 0.55%.  Let's now explore a few other paremters to tune using K = 48 $\pm$ 2.  The other parameters we'll look at are the algorithm types: "Brute Force, K-D Tree, and Ball Tree;" and leaf size.

In [ ]:
%%time
KNNdfs = pd.DataFrame(index = list( range(46, 50)),
                      columns = ["brute", "kd_tree", "ball_tree"],
                      dtype = "float")

# Iterate through K = 48 +/- 2
for K in list(range(46, 51)):
    # Then iterate through each algorithm
    for algorithm in list(KNNdfs.columns):
        clf = KNeighborsClassifier(n_neighbors = K, algorithm = algorithm)
        clf.fit(X_train, y_train)
        yhat= clf.predict(X_test)
        accuracy = mt.accuracy_score(y_test, yhat) * 100
        KNNdfs.loc[K, algorithm] = accuracy

# Output the results
print(KNNdfs)

What we can see is that the highest accuracy comes from 48 neighbors regardless of the type of algorithm we use.  Before we say every method is the same however, let's take a moment to explore the leaf size for KD-Tree and Ball Tree algorithms.

In [ ]:
%%time
# Make a new multiIndex dataframe
arrays = [[46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46,
           47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47,
           48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48,
           49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49,
           50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50],
          [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
           25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
           25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
           25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
           25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]]
KNNdfs = pd.DataFrame(index = arrays,
                      columns = ["kd_tree", "ball_tree"],
                      dtype = "float")

# Iterate through each K value
for K in list( range(46, 51)):
    # Iterate through each leaf size
    for leaves in list(range( 25, 36)):
        # Then iterate through each algorithm
        for algorithm in list(KNNdfs.columns):
            clf = KNeighborsClassifier(n_neighbors = K, leaf_size = leaves, algorithm = algorithm)
            clf.fit(X_train, y_train)
            yhat= clf.predict(X_test)
            accuracy = mt.accuracy_score(y_test, yhat) * 100
            KNNdfs.loc[(K,  leaves), algorithm] = accuracy

In [ ]:
print("KD Tree best K: ", KNNdfs.kd_tree.idxmax()[0], 
      "\nBest leaf size: ", KNNdfs.kd_tree.idxmax()[1],
      "\nBest accuracy: ", round(KNNdfs.loc[KNNdfs.kd_tree.idxmax(), "kd_tree"], 2))

print("\nBall Tree best K: ", KNNdfs.ball_tree.idxmax()[0], 
      "\nBest leaf size: ", KNNdfs.ball_tree.idxmax()[1],
      "\nBest accuracy: ", round(KNNdfs.loc[KNNdfs.ball_tree.idxmax(), "ball_tree"], 2))

The best K value is still 48, but KD Tree and Ball Tree appear to be at a tie for accuracy with a leaf size of 25.  We'll restrict the model to just K = 48 now and test smaller leaf sizes to see if it makes any difference.

In [ ]:
%%time
# Make a new multiIndex dataframe
KNNdfs = pd.DataFrame(index = list(range( 1, 26)),
                      columns = ["kd_tree", "ball_tree"],
                      dtype = "float")

# Iterate through every leaf size
for leaves in list(range( 1, 26)):
    # Then iterate through each algorithm
    for algorithm in list(KNNdfs.columns):
        clf = KNeighborsClassifier(n_neighbors = 48, leaf_size = leaves, algorithm = algorithm)
        clf.fit(X_train, y_train)
        yhat= clf.predict(X_test)
        accuracy = mt.accuracy_score(y_test, yhat) * 100
        KNNdfs.loc[leaves, algorithm] = accuracy

In [ ]:
print("KD Tree best leaf size: ", KNNdfs.kd_tree.idxmax(),
      "\nBest accuracy: ", round(KNNdfs.loc[KNNdfs.kd_tree.idxmax(), "kd_tree"], 2))

print("\nBall Tree best leaf size: ", KNNdfs.ball_tree.idxmax(),
      "\nBest accuracy: ", round(KNNdfs.loc[KNNdfs.ball_tree.idxmax(), "ball_tree"], 2))

So the KNN model performs the same under the Brute Force algorithm, KD Tree, or Ball Tree and is unaffected by the leaf size.  It would seem the only parameter that makes any signficant difference is the value for K, and even then, the difference is marginal compared to other K values.  

Nonetheless, our KNN model will use 48 neighbors, 1 for the leaf size, and the algorithm set to the default value, 'auto.'  Using these values, we will now examine the precision of the model.

In [26]:
# Set the parameters as described above
clf=KNeighborsClassifier(n_neighbors = 48, leaf_size = 1, algorithm = 'auto')
clf.fit(X_train, y_train)
yhat= clf.predict(X_test)
train_predt= clf.predict(X_train)

# Define some helper functions
def per_class_accuracy(ytrue, yhat):
    conf = mt.confusion_matrix(ytrue, yhat)
    norm_conf = conf.astype('float') / conf.sum(axis = 1)[:, np.newaxis]
    return np.diag(norm_conf)

def plot_class_acc(ytrue, yhat, title = ''):
    acc_list = per_class_accuracy(ytrue, yhat)
    plt.bar(range(len(acc_list)), acc_list)
    plt.xlabel('Classification')
    plt.ylabel('Accuracy within class')
    plt.title(title + ", Total Acc = %.1f"%(100 * mt.accuracy_score(ytrue, yhat)))
    plt.grid()
    plt.ylim([0, 1])
    plt.xticks([0, 1, 2 ,3], ["Commercial", "Manufacturing", "Park", "Residential"])
    plt.show()
    
# Display the plot
plot_class_acc(y_test, yhat, title="KNN model")

We see that this model is especially good at classifying Residential zoning districts and not so great at classifying Commercial or Manufacturing districts.  Our favorite confusion matrix figure below illustrates the distribution of predictions.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(WIDTH, HEIGHT))
cbar_ax = fig.add_axes([.91, .3, .03, .4])
ax[0].set_title('Training Data Results', fontdict={'size': 20})
ax[1].set_title('Testing Data Results', fontdict={'size': 20})
fig.text(.47, .25, 'New Zone District Predicted label', ha='center', va='center', fontdict={'size': 15})
fig.text(-.03, 0.5, 'New Zone District True Label', ha='center', va='center', rotation='vertical', fontdict={'size': 15})
# Labels for "Y" and "N" don't seem to load either.  Tried many different ways.
labels = le.classes_
print(labels)
mat1 = confusion_matrix(y_train, train_predt)/len(y_train)*100
mat2 = confusion_matrix(y_test, yhat)/len(y_test)*100
ax[0] = sns.heatmap(mat1, 
            square=True, 
            annot=True,
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[0])
ax[0].set_xticklabels((labels))
ax[0].set_yticklabels((labels))
ax[1] = sns.heatmap(mat2, 
            square=True, 
            annot=True, 
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[1])
ax[1].set_xticklabels((labels))
ax[1].set_yticklabels((labels))
fig.tight_layout(rect=[0, 0, .9, 1])
plt.show()

As expected, the model appears to be heavily biased into predicting Residential zone districts versus any other classification.  This is likely due to the shear volume of observations that have this classification.  Although we retain the same percentage of each label in the stratified shuffle split, there are still many many more instances of residentially zoned lots.  The KNN model at its best produced a precision of just over .72 for the Residential classification.  Parks were just under .53 precision.

One of the key advantages of the KNN model is its flexibility.  We explored different K values using different algorithms and leaf size.  The unfortunate down side of our data is that these other parameters did not seem to make a difference in the evaluation metric, accuracy, we were interested in.

Interpreting feature importance of a KNN model is not intuitive.  Since an observation is classified according to the distance metric of the K nearest neighbors, there is no obvious way to extract this information in the case of a specific class.  We could however, examine the classifications for the K nearest neighbors around a specific observation, but that provides little insight to the importance of individual features.

We leave the KNN model for now and look towards XGBoost.

#### XGBooster Model

The open-source XGBoost algorithym was developed by Tianqi Chen. It has become a widely used tool among Data Scientists.  Its name stands for eXtreme Gradient Boosting; and it is capable of performing the three main forms of gradient boosting (Gradient Boosting (GB), Stochastic GB and Regularized GB).

Boosting is an ensemble method which creates a strong classifier based on "weak" classifiers.  Weak and strong refer to a how correlated are the learners to a target variable, meaning that the errors of the previous model are corrected by the next predictor, iteratively speaking. As opposed to AdaBoost, where a weight is added to the weak learners after iteration, XGBoost uses gradient descent, i.e., the method fits a new model to the new residuals of the previous prediction.

Our XGBooster model will use the same split that we leveraged earlier via the Stratified Shuffle Split function from sklearn.  The parameters we used are: 5 split iterations utilizing 50% of the labels from each category to maintain an apples to apples comparison between the models.  Later, we'll modify the split between test and training data to 20% and 80% and raise the splitting iterations to 10 to see how this will impact the performance of the model.  Once again, we'll be examining the model's accuracy and precision as our evaluation metrics.  This way we have something to compare against the previous models.

In [ ]:
%%time
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.5, train_size=0.5)

le = preprocessing.LabelEncoder()
nzd = le.fit_transform(nyc_2013.new_zone_dist)

for trainidx, testidx in cv.split(X, nzd ):
    X_train  = X.iloc[trainidx] 
    X_test   = X.iloc[testidx] 
    y_train  =  nzd[trainidx]
    y_test   =  nzd[testidx]


our_xgbooster = xgb.XGBClassifier(max_depth = 5)
our_xgbooster.fit(X_train, y_train)
strong_accuracy_test = our_xgbooster.score(X_test, y_test)
print("XGBooster Strong Accuracy at 50/50 CV split: ", strong_accuracy_test)

In [ ]:
%%time
cv = StratifiedShuffleSplit(n_splits = 10, test_size = 0.2, train_size=0.8)

le = preprocessing.LabelEncoder()
nzd = le.fit_transform(nyc_2013.new_zone_dist)

for trainidx, testidx in cv.split(X, nzd ):
    X_train  = X.iloc[trainidx] 
    X_test   = X.iloc[testidx] 
    y_train  =  nzd[trainidx]
    y_test   =  nzd[testidx]


our_xgbooster = xgb.XGBClassifier(max_depth = 5)
our_xgbooster.fit(X_train, y_train)
strong_accuracy_test = our_xgbooster.score(X_test, y_test)
print("XGBooster Strong Accuracy at 20/80 CV split:", strong_accuracy_test)

As shown above, using the 80/20 split improved the accuracy of the model from 95.96% to 96.13, but the execution turn-around-time (TAT) incremented about 1 minute from 2 to 3.5 minutes.  It does beg the question if we should have been using 80/20 for all of our previous models, or if this slight increase in accuracy is specific to the XGBoost model.

We also reduced the parameter max_depth to 5 from the default which is 6 to control the maximum depth of a tree and avoid overfitting.  We tested the XGBooster model with a max_depth of 7 to gauge the performance and accuracy when raising this parameter value; accuracy was reduced and TAT went up to 4+ minutes (code not shown).

In [ ]:
#XGBooster Predictions
XGBooster_test_predictions  = pd.DataFrame(our_xgbooster.predict(X_test))
XGBooster_train_predictions = pd.DataFrame(our_xgbooster.predict(X_train))

# Define the figure
fig, ax = plt.subplots(1, 2, figsize=(WIDTH, HEIGHT))
cbar_ax = fig.add_axes([.91, .3, .03, .4])
ax[0].set_title('XGB Training Data Results', fontdict={'size': 20})
ax[1].set_title('XGB Testing Data Results', fontdict={'size': 20})
fig.text(.47, .25, 
         'New Zone District Predicted label', 
         ha = 'center', 
         va = 'center',
         fontdict = {'size': 15})
fig.text(-.03, 0.5, 
         'New Zone District True Label',
         ha = 'center',
         va = 'center',
         rotation = 'vertical',
         fontdict = {'size': 15})
labels = le.classes_
mat1 = confusion_matrix(y_train, XGBooster_train_predictions) / len(y_train) * 100
mat2 = confusion_matrix(y_test, XGBooster_test_predictions) / len(y_test) * 100
ax[0] = sns.heatmap(mat1, 
            square=True, 
            annot=True,
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[0])
ax[0].set_xticklabels((labels))
ax[0].set_yticklabels((labels))

ax[1] = sns.heatmap(mat2, 
            square = True, 
            annot = True, 
            fmt = '.2g',
            cbar_ax = cbar_ax,
            linewidths = .5, 
            ax = ax[1])
ax[1].set_xticklabels((labels))
ax[1].set_yticklabels((labels))
fig.tight_layout(rect=[0, 0, .9, 1])

Our XGBooster model performance for classifying New Zone District was highly accurate with a accuracy score of 96.13%. It is worth mentioning that the model took a significantly shorter time to achieve this level of accuracy when compared to Logistic Regression and KNN.

The confusion matrices of the predictions of the testing and training data are shown above.  In section 3.2.1 we calculated the distribution of the New Zone District feature in the data set that had the following results:

1. Residential:  6858  or 59.66 %
2. Park:  2551  or 22.19 %
3. Manufacturing:  1281  or 11.14 %
4. Commercial:  806  or 7.01 %

Comparing the results to the confussion matrix's results on our testing data, we see that the results are very close to the original classification in the data set, as follows:

1. Residential:    59 %
2. Park:           22 %
3. Manufacturing:  11 %
4. Commercial:     4.4 %

Clearly, the XGBoost model is superior to LR and KNN for our data set.  Couple that with the simple fact that it processed so quickly makes it an obvious choice for future models.  We only explored one parameter in the XGBoost model (max depth), but we're confident that we can delve into all of the optional parameters and find an even better fit given more time.  Because of these highly accurate results, we will examine XGBoost in the next classification task as well for Assessed Land Value Range.  Before we get there though, let's look at feature importance for this model.

In [ ]:
# Get the weights for all features
impt_features = our_xgbooster.feature_importances_

# The top 10 features are where impt_features are sorted in descending order
fig = plt.figure(figsize = (WIDTH, HEIGHT))
top_impt = impt_features[impt_features.argsort()[-10:][::-1]]
plt.bar(range(len(top_impt)), top_impt)
plt.xticks(range(0,10), list( X_train.columns[i] for i in impt_features.argsort()[-10:][::-1]), rotation = 90)
plt.title("Feature Importance")
plt.show()

Here we can see how just the top 10 features weigh in on determining the classification for the XGBoost model.  At rank 1 we see Land Use Category 9: Open Space & Outdoor Recreation, followed by Building Class Q: Outdoor Recreation Facilities.  Combined, these two features can split the data to confidently classify lots which are parks and which are not.  Further down we see Land Use Category 5: Commercial & Office Buildings.  It seems as though the Allowable Building to Floor Area would then be used to separate Commercial from Residential, and Land Use Category 5 solidifies the decision.  It's curious to see two Fire Companies listed in the top 10.  Perhaps they cover a large area, devoid of diversity in zoning districts?  In any case, we see the numerical value of the top 10 weights below.

In [ ]:
XG_weights = pd.DataFrame(top_impt, 
                          index = list( X_train.columns[i] for i in impt_features.argsort()[-10:][::-1]),
                          columns = ["Weight"])

XG_weights

The first 2 features are orders of magnitude greater than the rest with the exception of Allowable Building to Floor Area (rank 3).  These numbers, in conjunction with the bar chart above illustrate just how important those features are compared to the rest.  At a 96% accuracy, this model has done the best job so far interpreting feature importance.

The two models that can derive variable importance only had one variable in common and that was land_use_category_9. This tells us that based on the model that we choose, we will need to find different methods to apply it out in the field.

### Classification Task for Assessed Land Value Range

This portion is concerned with the creation and manipulation of models designed to classify the Assessed Land Value Range feature.  For this attribute, we will build models using Random Forest, Naive Bayes, and XGBoost once again.  The first step however, is to get our training data into a usable format.

In [ ]:
# Output the distribution of Assess Land Value labels
for i in list(range(0,3)):
    print(nyc_2013.assess_range.value_counts().index[i] + ": ",
          nyc_2013.assess_range.value_counts()[2-i], 
          " or",
          round(nyc_2013.assess_range.value_counts()[2-i]
                /nyc_2013.assess_range.value_counts().sum()
                *100, 2),
          "%")

The data is much more evenly spread across the 3 levels in the feature. This bodes well for modeling purposes. We now split the data into testing and training as described above.

In [ ]:
# Identify categorical variables.  Assessed Land Value must also be excluded since it completely determines the
# target variable Assessed Land Value Range
categorical_fields = list(nyc_2013.drop(
    columns = ["assess_range"]).select_dtypes(
    include=['category']).columns)

# Creating dummy variables for their levels
X = pd.get_dummies(nyc_2013, columns = categorical_fields).drop(
    columns = ["assess_range", "assess_land"])
    
print("Shape after one-hot encoding categorical variables: ", X.shape)

#### Random Forest Model

The Random Forest (RF) model will use the same split as before (Stratified Shuffle Split) with 5 splits utilizing 50% of the labels from each category.  Accuracy and precision will be our evaluation metrics.  We expect RF to perform very well in both of these areas.

In [ ]:
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.5, train_size=0.5)

le = preprocessing.LabelEncoder()
alr = le.fit_transform(nyc_2013.assess_range)

for trainidx, testidx in cv.split(X, nzd):
    # note that these are sparse matrices
    X_train = X.iloc[trainidx] 
    X_test = X.iloc[testidx] 
    y_train = alr[trainidx]
    y_test = alr[testidx]

In [ ]:
%%time
clf=RandomForestClassifier(max_depth=50, n_estimators=150, n_jobs=-1)
clf.fit(X_train, y_train)
yhat= clf.predict(X_test)
total_accuracy = mt.accuracy_score(y_test, yhat)
print ('Accuracy', total_accuracy)

Incredibly, on the first run the accuracy of this model was over 90% and computed in under 2 seconds!  It may be prudent to find the accuracy ratio per class of the Assessed Land Value Range.  High accuracy and low false positive ratios for each class are what make this model a good fit considering slightly unbalanced data between the different number of observation per level in this feature.

In [ ]:
def per_class_accuracy(ytrue, yhat):
    conf = mt.confusion_matrix(ytrue, yhat)
    norm_conf = conf.astype('float') / conf.sum(axis = 1)[:, np.newaxis]
    return np.diag(norm_conf)

def plot_class_acc(ytrue, yhat, title=''):
    acc_list = per_class_accuracy(ytrue, yhat)
    plt.bar(range(len(acc_list)), acc_list)
    plt.xlabel('Classification')
    plt.ylabel('Accuracy within class')
    plt.title(title + ", Total Acc = %.1f"%(100*mt.accuracy_score(ytrue, yhat)))
    plt.grid()
    plt.ylim([0,1])
    plt.xticks([0, 1, 2], ["Low", "Medium", "High"])
    plt.show()

In [ ]:
plot_class_acc(y_test, yhat, title = "Random Forest, Raw")

Overall, the accuracy of every classification is high, but we see that the "high" classification has the lowest accuracy of the 3.  This may be due to the simple fact that this label had the most number of observations between the other two.

We now examine our familiar confusion matrix to understand the individual predictions better.

In [ ]:
train_predt= clf.predict(X_train)

In [ ]:
# Define the figure
fig, ax = plt.subplots(1, 2, figsize=(WIDTH, HEIGHT))
cbar_ax = fig.add_axes([.91, .3, .03, .4])
ax[0].set_title('Training Data Results', fontdict={'size': 20})
ax[1].set_title('Testing Data Results', fontdict={'size': 20})
fig.text(.47, .25, 'Assess Range Predicted label', ha='center', va='center', fontdict={'size': 15})
fig.text(-.03, 0.5, 'Assess Range True Label', ha='center', va='center', rotation='vertical', fontdict={'size': 15})
# Labels for "Y" and "N" don't seem to load either.  Tried many different ways.
labels = le.classes_
mat1 = confusion_matrix(y_train, train_predt)/len(y_train)*100
mat2 = confusion_matrix(y_test, yhat)/len(y_test)*100
ax[0] = sns.heatmap(mat1, 
            square=True, 
            annot=True,
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[0])
ax[0].set_xticklabels((labels))
ax[0].set_yticklabels((labels))
ax[1] = sns.heatmap(mat2, 
            square=True, 
            annot=True, 
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[1])
ax[1].set_xticklabels((labels))
ax[1].set_yticklabels((labels))
fig.tight_layout(rect=[0, 0, .9, 1])
plt.show()

In the training set, 100% precision is obtained for the "High" classification, indicating the data may be overfitted.  But when we examine the test data, the model doesn't perform quite as well as in the training data, lending credence to the idea that the data may be overfitted. Again, the overall accuracy for the test set was 90.27%.

Next, let's see if we can identify which features were most important in classifying the Assessed Land Value Range.

In [ ]:
# out of 2023 features, there must be some unimportnat features for bulding final model, therefore the most
# improtant features can beindentified using the following histigram according to percent of importance.
impt_features = clf.feature_importances_
top_impt = impt_features[impt_features.argsort()[-10:][::-1]]
plt.bar(range(len(top_impt)), top_impt)
plt.xticks(range(0,10), list( X_train.columns[i] for i in impt_features.argsort()[-10:][::-1]), rotation = 90)
plt.title("Feature Importance")
plt.show()

Unspurprising, the single most important feature is the Total Area with Exempt Land, Lot Front, and Exempt Total following closely behind.

In [ ]:
RF_weights = pd.DataFrame(top_impt, 
                          index = ["Rank 1", "Rank 2", "Rank 3", "Rank 4", "Rank 5", 
                                     "Rank 6", "Rank 7", "Rank 8", "Rank 9", "Rank 10"],
                          columns = ["Weight"])
RF_weights

When trying to classify whether a given lot is going to fall into a particular range scale (Low, Medium, or High), it is reasonable to believe that features pertaining to the area of the land how much of it is taxed would be the greatest indicators for how to classify the lot.  We also Commercial Floor Area in the top 10 of important features analyzed when making this classification.  We know from our Data Preparation phase that commercially zone districts hold the largest percentage of "High" valued Assessed Land Range (greater than 66%) compared to the other zoning regions.  If a lot then is small, the model appears to placing some weight as to how much of the lot is for commercial use.  That could be the telltale sign that even though a lot is small, it gets the "High" classification for likely being commercially zoned.

#### Naive Bayes Model

The next model for analysis of the Assessed Land Value Range will be Naive Bayes (NB).  We will use the same split as before (Stratified Shuffle Split) with 5 splits utilizing 50% of the labels from each category.  As with Random Forest, we are interested in the accuracy of the model and the precision for each classification.  We feel these are important metrics because a high accuracy will tell us how viable our model is overall.  By examining precision, we can identify where our model falls short.

The documentation for sklearn's Naive Bayes model recommends trying both multinomial and Bernoulli if time permits.  Time permits, so we will show both.

In [ ]:
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.5, train_size=0.5)

le = preprocessing.LabelEncoder()
alr = le.fit_transform(nyc_2013.assess_range)

for trainidx, testidx in cv.split(X, nzd):
    # note that these are sparse matrices
    X_train = X.iloc[trainidx] 
    X_test = X.iloc[testidx] 
    y_train = alr[trainidx]
    y_test = alr[testidx]

In [ ]:
# alpha is the smoothing parameter.  It can be varied from 0 to 1.
print("Multinomial NB")
for alpha in np.arange(0.0, 1.1, 0.1):
    clf_mnb = MultinomialNB(alpha = alpha)
    clf_mnb.fit(X_train, y_train)
    y_hat = clf_mnb.predict(X_test)
    accuracy = accuracy_score(y_test, y_hat) * 100
    print("Alpha: ", round(alpha, 1), "\tAccuracy:" , round(accuracy, 6))

print("\nBernoulli NB")
for alpha in np.arange(0.0, 1.1, 0.1):
    clf_bnb = BernoulliNB(alpha = alpha)
    clf_bnb.fit(X_train, y_train)
    y_hat = clf_bnb.predict(X_test)
    accuracy = accuracy_score(y_test, y_hat) * 100
    print("Alpha: ", round(alpha, 1), "\tAccuracy:" , round(accuracy, 6))

Immediately we can see that the Bernoulli NB model outsines the Multinomial model.  While the accuracies do not change much within the two models, we do see that the best accuracy comes from an $\alpha$ value of 0.1.  This tells us that a very small amount of Lidstone smoothing allows for a more accurate model.  The Bernoulli model is a better choice since most of our features are binary after one-hot encoding.  Bernouilli models excel when that is the case.  Going forward, we will use the Bernoulli NB model with $\alpha$ = 0.1 as we adjust one more paremeter, fit_prior, to try to obtain even better results.

In [ ]:
# Default value for fit_prior is true.  We'll try it with it set to false.
clf_bnb_fpf = BernoulliNB(alpha = .1, fit_prior = False)
clf_bnb_fpf.fit(X_train, y_train)
y_hat = clf_bnb_fpf.predict(X_test)
accuracy = accuracy_score(y_test, y_hat) * 100
print("fit_prior = Flase \tAccuracy: ", round(accuracy, 6))

clf_bnb = BernoulliNB(alpha = .1, fit_prior = True)
clf_bnb.fit(X_train, y_train)
y_hat = clf_bnb.predict(X_test)
accuracy = accuracy_score(y_test, y_hat) * 100
print("fit_prior = True \tAccuracy: ", round(accuracy, 6))

The difference isn't huge, but it does seem that allowing the model to learn prior class probabilities slightly increases the accuracy.  Let's see which features carry the most weight for this model.

In [ ]:
# Get the weights for all features
impt_features = clf_bnb.coef_

# Create a placeholder for just the top n most important
n_features = 20
top_impt = np.zeros( (impt_features.shape[0], n_features), dtype = 'float')
indices = np.zeros( (impt_features.shape[0], n_features), dtype = 'int')

# Define the figure
fig, ax = plt.subplots(1, 3, figsize=(WIDTH, HEIGHT))

# Extract the relevant data
for i in range(0, impt_features.shape[0]):
    indices[i,:] = impt_features[i, :].argsort()[:n_features]
    top_impt[i, :] = impt_features[i, indices[i, :]]
    #plt.xticks(range(0,10), list( X_train.columns[i] for i in impt_features[i, :].argsort()[:10]), rotation = 90)

# Plot it on a subplot
ax[0].set_title('High Features', fontdict={'size': 20})
ax[0].bar(range(0, len(top_impt[0, :])), top_impt[0, :])
ax[0].xaxis.set_ticks(np.arange(0, n_features))
ax[0].set_xticklabels( list( X_train.columns[i] for i in indices[0, :]), rotation = 90)
ax[1].set_title('Low Features', fontdict={'size': 20})
ax[1].bar(range(len(top_impt[1, :])), top_impt[1, :])
ax[1].xaxis.set_ticks(np.arange(0, n_features))
ax[1].set_xticklabels( list( X_train.columns[i] for i in indices[1, :]), rotation = 90)
#fig.subplots_adjust(hspace = .6)
ax[2].set_title('Medium Features', fontdict={'size': 20})
ax[2].bar(range(len(top_impt[2, :])), top_impt[2, :])
ax[2].xaxis.set_ticks(np.arange(0, n_features))
ax[2].set_xticklabels( list( X_train.columns[i] for i in indices[2, :]), rotation = 90)


plt.show()

The only classification with any discernible difference in the top 20 features is for the "High" land value range.  We see for that category, Building Class U sits at rank 1, although the next 14 features appear to have nearly the same (if not the same) value.  Building Class U is for Utility Bureau Properties.  It surprises me why this building class carries so much weight in the decision process for High valued lots and definitely raises a few questions.  We also see Land Use Category 1 (for One & Two Family Buildings) present at rank 3.  This one makes sense to me though since "Low" Assessed Value constitutes the majority of data for Residential Zoning Districts.  Overwhelmingly present in the most important features are the various Fire Company codes.  We are puzzled why this feature is so significant in classifying the Assessed Land Value Range and would need to conduct addtional exploratory data analyses, possibly with the appropriate shapefiles for Fire Company, to understand how this feature became to be so important in the Naive Bayes model.

Below we examine the actual values of the weights for the top 20 features.

In [ ]:
NB_weights = pd.DataFrame(top_impt, 
                          index = ["High", "Low", "Medium"],
                          columns = ["Rank 1", "Rank 2", "Rank 3", "Rank 4", "Rank 5", 
                                     "Rank 6", "Rank 7", "Rank 8", "Rank 9", "Rank 10",
                                     "Rank 11", "Rank 12", "Rank 13", "Rank 14", "Rank 15",
                                     "Rank 16", "Rank 17", "Rank 18", "Rank 19", "Rank 20"])
NB_weights.transpose()

What we can see is that most of the weights share the same value.  It isn't until we look well beyond the first 20 that we start to see any noticable difference.  Below is the same plot from above, but looking at all features.

In [ ]:
# Get the weights for all features
impt_features = clf_bnb.coef_

# Create a placeholder for just the top n most important
n_features = impt_features.shape[1]
top_impt = np.zeros( (impt_features.shape[0], n_features), dtype = 'float')
indices = np.zeros( (impt_features.shape[0], n_features), dtype = 'int')

# Define the figure
fig, ax = plt.subplots(1, 3, figsize=(WIDTH, HEIGHT))

# Extract the relevant data
for i in range(0, impt_features.shape[0]):
    indices[i,:] = impt_features[i, :].argsort()[:n_features]
    top_impt[i, :] = impt_features[i, indices[i, :]]

# Plot it on a subplot
ax[0].set_title('High Features', fontdict={'size': 20})
ax[0].bar(range(0, len(top_impt[0, :])), top_impt[0, :])
ax[1].set_title('Low Features', fontdict={'size': 20})
ax[1].bar(range(len(top_impt[1, :])), top_impt[1, :])
ax[2].set_title('Medium Features', fontdict={'size': 20})
ax[2].bar(range(len(top_impt[2, :])), top_impt[2, :])


plt.show()

The gaps near 100, 300, and 500 appear to be artifacts of the plotting method and do not reflect 0 values for features in that region.  The big takeaway here though is that the NB model assigns the same weights to tens or even hundreds of features when making a classification.  It isn't until near the 400th most important feature that weights start to vary from feature to feature going forward.

While this model isn't very accurate, it does compute relatively quickly.  The Bernoulli NB was a little slower than the Multinomial, but the difference still warrants the use of Bernoulli.  Since much of our data was binarized due to one-hot encoding, Bernoulli is a much better choice than Multinomial anyway.  With some further study, we may be able to do more with this model knowing that so many of the features carry the same weight.  For now though, we'll look at another model.

#### XGBooster Model

Our XGBooster model to predict Assessed Land Value Range will use the same split as we leveraged earlier via the Stratified Shuffle Split function from sklearn. We continue to use this split for consistency between models and to evaluate how each model performs under the same data split.  Again, the parameters we used are: 5 split iterations utilizing 50% of the labels from each category.

As shown below, the model performed at an accuracy of 0.99965, and the execution TAT, including the confussion matricess remained at 25.3 seconds.

We kept the max_depth parameter at 5, one level under the default which is 6 since max_depth is to control the maximum depth of a tree to avoid overfitting. We tested earlier with other values and found that execution time goes up without any gain in the accuracy metric.

In [ ]:
# Split the data
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.5, train_size=0.5)

# Encode the labels
le = preprocessing.LabelEncoder()
alr = le.fit_transform(nyc_2013.assess_range)

for trainidx, testidx in cv.split(X, alr ):
    X_train  = X.iloc[trainidx] 
    X_test   = X.iloc[testidx] 
    y_train  =  alr[trainidx]
    y_test   =  alr[testidx]

# Define the model
alr_xgbooster = xgb.XGBClassifier(max_depth=5)
alr_xgbooster.fit(X_train, y_train)

# Output the accuracy
strong_accuracy_test = alr_xgbooster.score(X_test,y_test)
print("ALR XGBooster Strong Accuracy at 50/50 CV split: ", strong_accuracy_test)


# ALR XGBooster Predictions
XGBooster_test_predictions  = pd.DataFrame(alr_xgbooster.predict(X_test))
XGBooster_train_predictions = pd.DataFrame(alr_xgbooster.predict(X_train))


#Confusion Matrices
# Define the figure
fig, ax = plt.subplots(1, 2, figsize=(WIDTH, HEIGHT))
cbar_ax = fig.add_axes([.91, .3, .03, .4])
ax[0].set_title('XBG Training Data Results', fontdict={'size': 20})
ax[1].set_title('XBG Testing Data Results', fontdict={'size': 20})
fig.text(.47, .25, 'Assess Range Predicted label', ha='center', va='center', fontdict={'size': 15})
fig.text(-.03, 0.5, 'Assess Range True Label', ha='center', va='center', rotation='vertical', fontdict={'size': 15})
# Labels for "Y" and "N" don't seem to load either.  Tried many different ways.
labels = le.classes_
mat1 = confusion_matrix(y_train, XGBooster_train_predictions)/len(y_train)*100
mat2 = confusion_matrix(y_test, XGBooster_test_predictions)/len(y_test)*100

ax[0] = sns.heatmap(mat1, 
            square=True, 
            annot=True,
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[0])
ax[0].set_xticklabels((labels))
ax[0].set_yticklabels((labels))
ax[1] = sns.heatmap(mat2, 
            square=True, 
            annot=True, 
            fmt='.2g',
            cbar_ax = cbar_ax,
            linewidths=.5, 
            ax= ax[1])
ax[1].set_xticklabels((labels))
ax[1].set_yticklabels((labels))
fig.tight_layout(rect=[0, 0, .9, 1])
plt.show()

## Deployment

* **[5 points]** _How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?_

Using our model, most notabley the XGBoosted Classifier with a very high accuracy between both variables, we can help clients in real estate development and commercial/residential contracting.

For a contractor interested in building in the New York City area, the zoning district feature is important in understanding the type of work that needs to be done at a specific site. Coupled with Land Assessment Range,  we can help contractors select the most profitable projects. The ability to predict the type fo zoning classification, and the range of the final assessment, contractors will be able effectively bid the correct amount on NYC jobs. We could measure successful bids over time as a way to assess the value of the model. Although there are other variables associated with wins and losses, it is a good proxy for selecting bids close to the winning price.

For clients interested in real estate development, we can also use the variable importance features to help assessors pinpoint the most important features of a property before sale. The company can make checklist for those out in the field to select the best properties. We can then measure the value of our model by the final sales bin price accuracy  with new data points. Although this could be regressed continuously, the range allows for people to make real world decisions and make bids purchases on real estate effectively.

In addition to the data provided by the city, we could collect data not directly related to individual properties. For example, crime statistics for residential areas might help accurately predict the Land Value.

For commercial zones we could collect more data on Permit Issuance Data to better understand the way the city give zoning permits. In addition, Violation data can help factor into making close to the correct land value range. With more datapoints and features, the bins for the values could be made slightly more granular to allow for a closer price when making an estimate of the price.



Source: https://opendata.cityofnewyork.us

We could deploy this model in the middle of a pipeline. The accessors out in the field and contractors looking for new jobs, collect live data based on the top features that are selected in the original model through variable importance measures. we can then stage the data to be run through the model regularly. We could then find the average time to complete, or average time to sell and refresh our model based on the new data. This will allow us to see bottom of the funnel of results with a dynamic date refresh. This should increase accuracy over time.

In [ ]:
img = Image.open('./Data/Deployment.png')
img

## Exceptional Work

* **[10 points]** _You have free reign to provide additional analyses.  One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?_